In [1]:
import os

os.makedirs(os.path.join('..', 'data'), exist_ok=True)
data_file = os.path.join('..', 'data', 'house_tiny.csv')
with open(data_file, 'w') as f:
    f.write('NumRooms,Alley,Price\n')  # 列名
    f.write('NA,Pave,127500\n')  # 每行表示一个数据样本
    f.write('2,NA,106000\n')
    f.write('4,NA,178100\n')
    f.write('NA,NA,140000\n')

In [5]:

# 如果没有安装pandas，只需取消对以下行的注释来安装pandas
# !pip install pandas
import pandas as pd
import os

data_file = os.path.join('..', 'data', 'house_tiny.csv')
data = pd.read_csv(data_file)
print(data)

   NumRooms Alley   Price
0       NaN  Pave  127500
1       2.0   NaN  106000
2       4.0   NaN  178100
3       NaN   NaN  140000


In [6]:
# pip install cls==0.17.6，其他版本会导致inputs.mean不能直接用，需要过滤非数字列。
inputs, outputs = data.iloc[:, 0:2], data.iloc[:, 2]
inputs = inputs.fillna(inputs.mean())
print(inputs)

   NumRooms Alley
0       3.0  Pave
1       2.0   NaN
2       4.0   NaN
3       3.0   NaN


In [7]:
# 根据列的值拆分列
# 对于inputs中的类别值或离散值，我们将“NaN”视为一个类别。 
# 由于“巷子类型”（“Alley”）列只接受两种类型的类别值“Pave”和“NaN”， pandas可以自动将此列转换为两列“Alley_Pave”和“Alley_nan”。
inputs = pd.get_dummies(inputs, dummy_na=True)
print(inputs)

   NumRooms  Alley_Pave  Alley_nan
0       3.0           1          0
1       2.0           0          1
2       4.0           0          1
3       3.0           0          1


In [14]:
import torch
# 将预处理后的数据集转换为张量格式。DataFrame->Numpy->tensor(PyTorch)
X = torch.tensor(inputs.to_numpy(dtype=float))
y = torch.tensor(outputs.to_numpy(dtype=float))
X, y

(tensor([[3., 1., 0.],
         [2., 0., 1.],
         [4., 0., 1.],
         [3., 0., 1.]], dtype=torch.float64),
 tensor([127500., 106000., 178100., 140000.], dtype=torch.float64))

In [8]:
def dropNaNumMaxCol(dataCSV):
    '''DataFrames类型参数，删除nan最多的列'''
    num= dataCSV.isna().sum() # isna返回表格，sum返回pandas Series类型，sum(axis=0)统计每一列中缺失值的数量
#     num.idxmax(): 这个方法用于找出 Series 中值最大的索引（即具有最大值的列名）
    print(num, num.shape)
    numDict= num.to_dict()
    print(numDict)
    maxKey =max(numDict,key=numDict.get) #取字典中最大值的键
    del dataCSV[maxKey] #删除缺失值最多的列（在原数据上操作）
    return dataCSV

In [9]:
dropNaNumMaxCol(data)
print(data)

NumRooms    2
Alley       3
Price       0
dtype: int64 (3,)
{'NumRooms': 2, 'Alley': 3, 'Price': 0}
   NumRooms   Price
0       NaN  127500
1       2.0  106000
2       4.0  178100
3       NaN  140000


In [32]:
'''isna():返回一个布尔型的 DataFrame，缺失值位置为 True。'''

'''sum():对布尔型 DataFrame 进行求和，得到每列缺失值的数量。返回的结果是一个 pandas Series列向量，索引是 dataCSV 的列名，
   值是该列中缺失值的数量。'''

'''to_dict():这一步将Series转换为字典类型,字典的键是Series的索引,也即 dataCSV 的列名，值是对应列中缺失值的数量。'''

'''max():key (可选): 用于指定一个函数，该函数将应用于 iterable 的每个元素，然后根据函数的返回值进行比较找出最大值。
   numDict.get:返回字典中指定键的值；max() 函数会根据 key 函数的返回值来确定最大元素。'''


'max():key (可选): 用于指定一个函数，该函数将应用于 iterable 的每个元素，然后根据函数的返回值进行比较找出最大值。\n   numDict.get:返回字典中指定键的值；max() 函数会根据 key 函数的返回值来确定最大元素。'

In [41]:
def dropNaNumMaxCol2(data):
    count = 0
    count_max = 0
    labels = ['NumRooms','Alley','Price']
    for label in labels:
        count = data[label].isna().sum()
        if count > count_max:
            count_max = count
            flag = label
#     data_new = data.drop(flag,axis=1)
    data_new = data.drop([flag],axis=1) # 创建了一个新的 DataFrame 对象
    return data_new

In [42]:
print(dropNaNumMaxCol2(data))
print(data)

   NumRooms   Price
0       NaN  127500
1       2.0  106000
2       4.0  178100
3       NaN  140000
   NumRooms Alley   Price
0       NaN  Pave  127500
1       2.0   NaN  106000
2       4.0   NaN  178100
3       NaN   NaN  140000


In [ ]:
'''Pandas 设计成可以对整个 DataFrame 或其部分（列、行等）进行灵活的操作，这种设计使得数据处理更加方便和高效。
   通过 isna() 方法，你可以快速检查整个 DataFrame 或 特定列 中缺失值的情况，从而进行进一步的数据清洗、填充或分析工作。'''

In [49]:
print(data.isna(), data.isna().shape, max(data.isna().sum()), '\n\n', data)
'''一个语句替代上面的函数'''
'''dropna的 thresh 参数表示每列中至少需要存在的非缺失值数量，保留>=thresh个非缺失值的列及保留非缺失值多的列，删除了缺失值过多的列'''
'''data.count() 是 Pandas 中用于统计 DataFrame 或 Series 中非缺失值（非 NaN 值）的数量的方法。此处统计DataFrame则返回Series。'''
'''下面语句保留了非缺失值数量>=(最小非缺失值数量+1)的列，删除了其他列。即保留了除最少非缺失值列外的所有列，即删除了最少非缺失值的列，
   即删除了最多缺失值NaN的列。'''
print(data.dropna(axis=1, thresh=(data.count().min()+1) ) )

   NumRooms  Alley  Price
0      True  False  False
1     False   True  False
2     False   True  False
3      True   True  False (4, 3) 3 

    NumRooms Alley   Price
0       NaN  Pave  127500
1       2.0   NaN  106000
2       4.0   NaN  178100
3       NaN   NaN  140000
   NumRooms   Price
0       NaN  127500
1       2.0  106000
2       4.0  178100
3       NaN  140000


In [58]:
print(data.count(), '\n', data.count().idxmax())
'''一个语句替代上面的函数'''
'''DataFrame.count() 不会统计每行的非缺失值数量，它只针对每列进行统计。
   如果你想要在原始 DataFrame 上进行修改并保持该修改，可以使用 inplace=True 参数，
   并且两者返回值不同。True则返回None,False则返回修改后的值而不会修改原数据。'''
print(data.drop(data.count().idxmax(), axis=1, inplace=False))

NumRooms    2
Alley       1
Price       4
dtype: int64 
 Price
   NumRooms Alley
0       NaN  Pave
1       2.0   NaN
2       4.0   NaN
3       NaN   NaN


In [ ]:
'''小结：
        - 删除数据：del,drop,dropna；
        - 查找NaN最多的列：data.count().idxmax()，thresh=(data.count().min()+1)，data[label].isna().sum()，data.isna().sum()；
        - axis=0行统计，axis=1列统计；
        - 可迭代对象Dictionary：max(numDict,key=numDict.get)'''